# 第 10 章 LangSmith - 追蹤程式的資料傳遞過程
在過去幾章，我們將各種函式和chain給串起來的時候，往往很難debug，這是因為這些資料在傳的時候，其資料結構並是我們方便讀取的東西。因此我們可以使用LangSmith來幫忙。LangSmith是一個平台，一旦程式中用了LangChain的東西，程式碼在把資料傳來傳去的時候上面就會記錄到底傳了哪些東西，方便我們debug。不過，LangSmith是要錢的。免費用戶每個月有5000條免費追蹤程式次數, 額外跟踪費用為0.005美元/次

In [1]:
import os

from rich import print as pprint

from langchain_community.vectorstores import Chroma
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tracers.context import tracing_v2_enabled
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['OPENAI_API_KEY'] = "sk-None-vowLahS2p4mOq6FP56VCT3BlbkFJTY1umKuhsfu61iHTNVDc"

## 10-1 如何使用LangSmith

In [3]:
chat_model = ChatOpenAI()
pprint(chat_model.invoke('妳好'))

AIMessage(
    content='你好，有什么可以帮助你的吗？',
    response_metadata={
        'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28},
        'model_name': 'gpt-3.5-turbo-0125',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-cb9dc325-9de1-4f0d-9010-0fe63b702b6a-0',
    usage_metadata={'input_tokens': 11, 'output_tokens': 17, 'total_tokens': 28}
)

接著到[LangSmith的工作站](https://smith.langchain.com/o/9445da78-d41c-50ad-bf39-69ffc50a5cf2/projects?paginationState=%7B%22pageIndex%22%3A0%2C%22pageSize%22%3A10%7D)查看dataflow。

![diagram](./Ch10/1.png)

### 10-2 流程鏈
使用LangChain內建的`tracing_v2_enabled`方法建立專案名稱，並到工作站查看。會發現多了一個"國家語言"專案

In [4]:
prompt = ChatPromptTemplate.from_template('在{city}講哪一種語言？')
str_parser = StrOutputParser()
chain = prompt | chat_model | str_parser

In [5]:
with tracing_v2_enabled(project_name="國家語言"):
    print(chain.invoke({"city":"東京"}))

在东京，主要使用的语言是日语。但是由于东京是国际化大都市，许多人也会说英语。此外，还有许多外国人在东京居住和工作，他们可能会说其他语言，如中文、韩语、西班牙语等。因此，东京是一个多语言环境的城市。


![diagram](./Ch10/2.png)

### 10-3 代理

In [6]:
class SearchRun(BaseModel):
    query: str = Field(description="給搜尋引擎的搜尋關鍵字, "
                                   "請使用繁體中文")

search_run = DuckDuckGoSearchRun(name="ddg-search",
                                 description="使用網路搜尋你不知道的事物",
                                 args_schema=SearchRun)

In [7]:
prompt = ChatPromptTemplate.from_messages([('system','你是一位好助理'),
                                           ('human','{input}'),
                                           MessagesPlaceholder(variable_name="agent_scratchpad")])

In [8]:
tools = [search_run]
agent = create_openai_tools_agent(llm=chat_model, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [9]:
with tracing_v2_enabled(project_name="搜尋工具"):
    result = agent_executor.invoke(
        {"input": "2023金馬獎影帝是誰?"})
    print(result['output'])

2023金馬獎影帝是吳慷仁，他憑藉電影《富都青年》獲得了最佳男主角獎。


![diagram](./Ch10/3.png)

### 10-4 RAG
使用第7章的資料庫

In [10]:
embeddings_model=OpenAIEmbeddings(model='text-embedding-3-large')
db = Chroma(persist_directory='./Ch7/db', embedding_function=embeddings_model)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

C:\Users\benny\anaconda3\envs\openai\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [11]:
str_parser = StrOutputParser()
template = ("""
    請根據以下內容加上自身判斷回答問題:\n
    {context}\n
    問題: {question}
    """)
prompt = ChatPromptTemplate.from_template(template)

In [12]:
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | chat_model | str_parser

In [13]:
with tracing_v2_enabled(project_name="PDF 問答"):
    print(chain.invoke("喝酒開車會被罰多少?"))

根據提供的文件內容，喝酒開車並且酒精濃度超過規定標準者，將會處以新臺幣30,000~120,000元罰鍰，並當場移置保管該汽車及吊扣其駕駛執照。


![diagram](./Ch10/4.png)